In [3]:
import numpy as np
import tensorflow as tf

from tensorflow.python.ops.rnn_cell import GRUCell
from tensorflow.python.ops.rnn_cell import LSTMCell
from tensorflow.python.ops.rnn_cell import MultiRNNCell
from tensorflow.python.ops.rnn_cell import DropoutWrapper, ResidualWrapper

from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.layers.core import Dense
from tensorflow.python.util import nest

from tensorflow.contrib.seq2seq.python.ops import attention_wrapper
from tensorflow.contrib.seq2seq.python.ops import beam_search_decoder

In [4]:
#Resetter
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [5]:
class Seq2SeqModel():
    def _init_(self,config,mode):
        assert mode.lower() in ['train','decode']
        
        self.config = config
        self.mode = mode.lower()
        
        self.cell_type = config['cell_type']
        self.hidden_units = config['hidden_units']
        self.depth = config['depth']
        self.attention_type = config['attention_type']
        self.embedding_size = config['embedding_size']
        
        self.num_encoder_symbols = config['num_encoder_symbols']
        self.num_decoder_symbols = config['num_decoder_symbols']
        
        self.use_residual = config['use_residual']
        self.attn_input_feeding = config['attn_input_feeding']
        self.use_dropout = config['use_dropout']
        self.keep_prob = 1.0 - config['dropout_rate']
        
        self.optimizer = config['optimizer']
        self.learning_rate = config['learning_rate']
        self.max_gradient_norm = config['max_gradient_norm']
        self.global_step = tf.Variable(0, trainable = Flase, name = 'global_step')
        self.global_epoch_step = tf.Variable(0,trainable=False, name = "global_epoch_step")
        self.global_epoch_step_op= tf.assign(self.global_epoch_step,self.global_epoch_step+1)
        
        self.dtype = tf.float16 if config['use_float16'] else tf.float32
        self.keep_prob_placeholder = tf.placeholder(self.dtype, shape=[], name = 'keep_prob')
        
        self.use_beamsearch_decode = False
        if self.mode == 'decode':
            self.beam_width = config['beam_width']
            self.use_beamsearch_decode = True if self.beam_width > 1 else False
            self.max_decode_step = config['max_decode_step']
        
        self.build_model()
        
    
        
    

In [6]:
    def build_model(self):
            print('building model..')

            #building encoder and decoder networks
            self.init_placeholder()
            self.build_encoder()
            self.build_decoder()
            self.summary_op = tf.summary.merge_all()

In [7]:
    def init_placeholders(self):
            #encoder inputs: [batch_size, max_time_steps]
            self.encoder_inputs = tf.placeholder(dtype = tf.int32, shape = (None,None), name = 'encoder_inputs')
            #encoder_inputs_length: [batch_size]
            self.encoder_inputs_length = tf.placeholder(dtype=tf.int32, shape=(None,) , name = 'encoder_inputs_length')
            
            #get dynamic batch_size
            self.batch_size = tf.shape(se,lf.encoder_inputs)[0]
            
            if(self.mode=='train'):
                
                #decoder_inputs: [batch_size,max_time_steps]
                self.decoder_inputs = tf.placeholder(dtype=tf.int32,shape=(None,None), name ='decoder_inputs')
                #decoder_inputs_length: [batch_size]
                self.decoder_inputs_length = tf.placeholder(dtype=tf.int32, shape=(None,), name='decoder_inputs_length')
                
                decoder_start_token=tf.ones(shape=[self.batch_size,1], dtype=tf.int32)*data_utils.start_token
                
                decoder_end_token=tf.ones(shape=[self.batch_size,1], dtype=tf.int32)*data_utils.end_token
            